# Calculate distance

In [18]:
from sedona.spark import *
from pyspark.sql.functions import col

In [2]:
# build a sedona session (sedona >= 1.4.1)
config = SedonaContext.builder(). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.4.1,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2'). \
    getOrCreate()

# create a sedona context
sedona = SedonaContext.create(config)

23/07/21 11:33:08 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
23/07/21 11:33:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/pengfei/opt/spark-3.3.0/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pengfei/.ivy2/cache
The jars for the packages stored in: /home/pengfei/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-00db68f1-eb92-4e4a-b255-17709ce066e5;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 222ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------------------------------------------------------------

23/07/21 11:33:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Euclidean distance between A and B

**ST_Distance (A:geometry, B:geometry)**

In [5]:
euclidean_dist_df=sedona.sql("SELECT ST_Distance(ST_GeomFromWKT('POINT (51.3168 -0.56)'), ST_GeomFromWKT('POINT (55.9533 -3.1883)'))")
euclidean_dist_df.show()

+-------------------------------------------------------------------------------------------------+
|st_distance(st_geomfromwkt(POINT (51.3168 -0.56), 0), st_geomfromwkt(POINT (55.9533 -3.1883), 0))|
+-------------------------------------------------------------------------------------------------+
|                                                                               5.3296428717128865|
+-------------------------------------------------------------------------------------------------+



## 2. Sphere distance

As we know the earth is not a 2D map, but a sphere. So the two point is in two different continant. The euclidean distance is no more accurate.

**ST_DistanceSphere (A:geometry, B:geometry, radius=6371008.0)** : It returns the `haversine / great-circle distance` of A using a given earth radius (default radius: 6371008.0). Unit is **meter**.

Compared to `ST_Distance + ST_Transform`, it works better for datasets that cover large regions such as continents or the entire planet. It is equivalent to PostGIS ST_Distance(geography, use_spheroid=false) and ST_DistanceSphere function and produces nearly identical results. It provides faster but less accurate result compared to ST_DistanceSpheroid.

> Geometry must be in EPSG:4326 (WGS84) projection and must be in lat/lon order. You can use ST_FlipCoordinates to swap lat and lon. For non-point data, we first take the centroids of both geometries and then compute the distance.

In [6]:
# We calculate the distance by using the default radius
sphere_dist_df1=sedona.sql("SELECT ST_DistanceSphere(ST_GeomFromWKT('POINT (51.3168 -0.56)'), ST_GeomFromWKT('POINT (55.9533 -3.1883)'))")
sphere_dist_df1.show()

+------------------------------------------------------------------------------------------------------------------+
|st_distancesphere(st_geomfromwkt(POINT (51.3168 -0.56), 0), st_geomfromwkt(POINT (55.9533 -3.1883), 0), 6371008.0)|
+------------------------------------------------------------------------------------------------------------------+
|                                                                                                 543796.9506134904|
+------------------------------------------------------------------------------------------------------------------+



In [7]:
# We calculate the distance by using a specific radius
sphere_dist_df2=sedona.sql("SELECT ST_DistanceSphere(ST_GeomFromWKT('POINT (51.3168 -0.56)'), ST_GeomFromWKT('POINT (55.9533 -3.1883)'),6378137.0)")
sphere_dist_df2.show()

+------------------------------------------------------------------------------------------------------------------+
|st_distancesphere(st_geomfromwkt(POINT (51.3168 -0.56), 0), st_geomfromwkt(POINT (55.9533 -3.1883), 0), 6378137.0)|
+------------------------------------------------------------------------------------------------------------------+
|                                                                                                 544405.4459192449|
+------------------------------------------------------------------------------------------------------------------+



## Distance between two polygon

The distance between two points is simple to calculate. How do we calculate the distance between two polygon.

The nature thinking is to first get the centroid of the polygon, then calculate the distance of the two centroids of the two polygons.

To simplify our life, the function ST_Distance can already calculate the distance of the two polygons directly

In [3]:
# get the spark context
sc = sedona.sparkContext

# set the encoding to utf8
sc.setSystemProperty("sedona.global.charset", "utf8")

In [4]:
countries_file_path = "/home/pengfei/data_set/kaggle/geospatial/countries_shape"

# read countries shape file
countries = ShapefileReader.readToGeometryRDD(sc, countries_file_path)
countries_df = Adapter.toDf(countries, sedona)
countries_df.createOrReplaceTempView("country")
countries_df.printSchema()

23/07/21 11:33:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
root
 |-- geometry: geometry (nullable = true)
 |-- featurecla: string (nullable = true)
 |-- scalerank: string (nullable = true)
 |-- LABELRANK: string (nullable = true)
 |-- SOVEREIGNT: string (nullable = true)
 |-- SOV_A3: string (nullable = true)
 |-- ADM0_DIF: string (nullable = true)
 |-- LEVEL: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- ADMIN: string (nullable = true)
 |-- ADM0_A3: string (nullable = true)
 |-- GEOU_DIF: string (nullable = true)
 |-- GEOUNIT: string (nullable = true)
 |-- GU_A3: string (nullable = true)
 |-- SU_DIF: string (nullable = true)
 |-- SUBUNIT: string (nullable = true)
 |-- SU_A3: string (nullable = true)
 |-- BRK_DIFF: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NAME_LONG: string (nullable = true)
 |-- BRK_A3: string (nullable = t

In [5]:
countries_df.show(1)

+--------------------+---------------+---------+---------+--------------------+------+--------+-----+-----------------+--------------------+-------+--------+--------------------+-----+------+--------------------+-----+--------+--------------------+--------------------+------+--------------------+-----------------+-------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+---------+---------+----------+----------+--------+-----------+--------+----------+--------+--------------------+--------------------+---------+--------+------+------+---------+------+-----+-----+-----+--------+---------+--------------------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------+----+--------+--------+---------+---------+----------+----------+--------------------+--------------------+-------

In [6]:
countries_df.select("geometry","NAME_EN").show(5)

+--------------------+--------------------+
|            geometry|             NAME_EN|
+--------------------+--------------------+
|MULTIPOLYGON (((3...|Zimbabwe         ...|
|MULTIPOLYGON (((3...|Zambia           ...|
|MULTIPOLYGON (((5...|Yemen            ...|
|MULTIPOLYGON (((1...|Vietnam          ...|
|MULTIPOLYGON (((-...|Venezuela        ...|
+--------------------+--------------------+
only showing top 5 rows



In [8]:
five_nearest_neighbor_df = sedona.sql("SELECT c.NAME_EN as county_name, ST_Distance(ST_PolygonFromEnvelope(1.0,100.0,1000.0,1100.0), c.geometry) AS distance FROM country as c ORDER BY distance DESC LIMIT 5")
five_nearest_neighbor_df.show(truncate=False)

+--------------------------------------------+------------------+
|county_name                                 |distance          |
+--------------------------------------------+------------------+
|Tonga                                       |211.3312856364424 |
|Wallis and Futuna                           |210.23925576273257|
|Niue                                        |208.17583859356532|
|Samoa                                       |206.72162008981843|
|American Samoa                              |206.13713647266056|
+--------------------------------------------+------------------+



In [9]:
fr_zone_file_path = "/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape"

# read countries shape file
fr_zones = ShapefileReader.readToGeometryRDD(sc, fr_zone_file_path)
fr_zones_df = Adapter.toDf(fr_zones, sedona)
fr_zones_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- insee: string (nullable = true)
 |-- nom: string (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- surf_ha: string (nullable = true)



In [10]:
fr_zones_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            geometry|               insee|                 nom|           wikipedia|             surf_ha|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|POLYGON ((9.32016...|2B222            ...|Pie-d'Orezza     ...|fr:Pie-d'Orezza  ...|     573.00000000...|
|POLYGON ((9.20010...|2B137            ...|Lano             ...|fr:Lano          ...|     824.00000000...|
|POLYGON ((9.27757...|2B051            ...|Cambia           ...|fr:Cambia        ...|     833.00000000...|
|POLYGON ((9.25119...|2B106            ...|Érone            ...|fr:Érone         ...|     393.00000000...|
|POLYGON ((9.28339...|2B185            ...|Oletta           ...|fr:Oletta        ...|    2674.00000000...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [20]:
fr_zones_df.count()

34955

In [11]:
fr_zones_df.createOrReplaceTempView("fr_zones")

In [17]:
# the gps coordinates for kremlin-Bicetre is 48.8100° N, 2.3539° E (from google maps)

KB_nearest_neighbor_df = sedona.sql("SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('48.8100,2.3539', ','), z.geometry) AS distance FROM fr_zones as z ORDER BY distance ASC LIMIT 10")
KB_nearest_neighbor_df.show(truncate=False)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|commune_name                                                                    |insee                                                                           |distance          |
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|Dzaoudzi                                                                        |97608                                                                           |1209337.6813830498|
|Pamandzi                                                                        |97615                                                                           |1210588.3239062466|
|Koungou                                                                         |976

The above result is clearly wrong, we need to find out why.

First, lets try to calculate the distance of two hospitals
The coordinates of `Bichat-Claude Bernard Hospital` is '48.8989,2.3318' (from google maps)

In [44]:
sedona.sql("SELECT ST_DistanceSphere(ST_PointFromText('48.8100,2.3539', ','), ST_PointFromText('48.8989,2.3318', ',')) AS distance ").show()

+-----------------+
|         distance|
+-----------------+
|6794135.740522429|
+-----------------+



The result returns by Google Maps is 11.4 km via real route. So we can consider the above result is correct. The problem is not the function `ST_DistanceSphere`.

Now lets check with a commune which is nearby the `kremlin-Bicetre`

In [29]:
ivry_df = fr_zones_df.filter(col("nom").startswith("Ivry-sur-Seine"))

In [30]:
ivry_df.show(2,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [46]:
ivry_df.createOrReplaceTempView("ivry")
sedona.sql("SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('48.8100, 2.3539', ','), z.geometry) AS distance FROM ivry as z ORDER BY distance ASC LIMIT 10").show()

+--------------------+--------------------+-----------------+
|        commune_name|               insee|         distance|
+--------------------+--------------------+-----------------+
|Ivry-sur-Seine   ...|94041            ...|6784236.321633434|
+--------------------+--------------------+-----------------+



The distance is 6784 kilometers, it's completely wrong.
My first guess is the projection system (epsg) which the French commune uses is not the same that the googl use. For example

- Google Earth is in a `Geographic coordinate system with the wgs84 datum`. (EPSG: 4326)

- Google Maps is in a `projected coordinate system that is based on the wgs84 datum`. (EPSG 3857)

- The data in Open Street Map database is stored in a gcs with units decimal degrees & datum of wgs84. (EPSG: 4326)

- The Open Street Map tiles and the WMS webservice, are in the projected coordinate system that is based on the wgs84 datum. (EPSG 3857)

As our data is from https://www.data.gouv.fr/fr/datasets/decoupage-administratif-communal-francais-issu-d-openstreetmap/. So yy first try is to convert the ivry-sur-seine polygon from epsg:4326 to 3857.

In [34]:
src_epsg = "EPSG:4326"
target_epsg = "EPSG:3857"

In [43]:
from sedona.sql.st_functions import ST_Transform

sedona.sql("SELECT ST_Transform(ivry.geometry,'epsg:4326','epsg:3857') FROM ivry").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

After conversion, the coordiantes is still wrong, because the above coordinates in google maps are not in france at all. My second try is to convert the hospital from 3847 to 4326. which is wrong too.

In [41]:
sedona.sql("SELECT ST_Transform(ST_PointFromText('48.8100,2.3539', ','), 'epsg:3857','epsg:4326')").show(truncate=False)

+------------------------------------------------------------------------------+
|st_transform(st_pointfromtext(48.8100,2.3539, ,), epsg:3857, epsg:4326, false)|
+------------------------------------------------------------------------------+
|POINT (0.0000211454434645 0.0004384676901787)                                 |
+------------------------------------------------------------------------------+



My third try is to inverse the order of 48.8100° N, 2.3539° E

In [47]:
sedona.sql("SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('2.3539, 48.8100', ','), z.geometry) AS distance FROM ivry as z ORDER BY distance ASC LIMIT 10").show()

+--------------------+--------------------+-----------------+
|        commune_name|               insee|         distance|
+--------------------+--------------------+-----------------+
|Ivry-sur-Seine   ...|94041            ...|3769.348960915047|
+--------------------+--------------------+-----------------+



This time the result is correct. So the conclusion is that the **OpenstreetMap which we download uses epsg:3857**. The coordinates of a Point has an order (E, N) (unlike the google order N, E).

Let's rerun the

In [48]:
KB_nearest_neighbor_df = sedona.sql("SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('2.3539,48.8100', ','), z.geometry) AS distance FROM fr_zones as z ORDER BY distance ASC LIMIT 10")
KB_nearest_neighbor_df.show(truncate=False)

+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|commune_name                                                                    |insee                                                                           |distance          |
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|Le Kremlin-Bicêtre                                                              |94043                                                                           |255.77950075329835|
|Gentilly                                                                        |94037                                                                           |1138.204118880015 |
|Villejuif                                                                       |940

Now the result is correct. Let's create a function which can return the 5 nearest commune of any hospital

In [55]:
def get_nearest_district_of_hospital(hospital_coordinates:str):
    query = f"SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('{hospital_coordinates}', ','), z.geometry) AS distance FROM fr_zones as z ORDER BY distance ASC LIMIT 10"
    print(query)
    return sedona.sql(query)


In [56]:
Bichat_Claude_Bernard = "2.3318, 48.8989"

In [58]:
bcb_nearest_commune = get_nearest_district_of_hospital(Bichat_Claude_Bernard)
bcb_nearest_commune.show(truncate=False)

SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('2.3318, 48.8989', ','), z.geometry) AS distance FROM fr_zones as z ORDER BY distance ASC LIMIT 10


+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|commune_name                                                                    |insee                                                                           |distance          |
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+------------------+
|Saint-Ouen-sur-Seine                                                            |93070                                                                           |1203.6350169395325|
|Clichy-la-Garenne                                                               |92024                                                                           |2930.2283994384406|
|Villeneuve-la-Garenne                                                           |920